<a href="https://colab.research.google.com/github/ellen107/bilingual_mnist/blob/main/bilingual.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import sys
import os
import cv2 as cv
import random
from pathlib import Path
import imageio
import skimage
import skimage.io
import skimage.transform 
import matplotlib.pyplot as plt
import scipy
from sklearn.model_selection import train_test_split
from sklearn import metrics
from tensorflow import keras
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPool2D, Dropout, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, Callback
from keras.utils import to_categorical
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
import tensorflow
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!git clone https://github.com/ellen107/bilingual_mnist.git

Cloning into 'bilingual_mnist'...
remote: Enumerating objects: 75037, done.
remote: Counting objects: 100% (75037/75037), done.
remote: Compressing objects: 100% (75034/75034), done.
remote: Total 75037 (delta 1), reused 75031 (delta 1), pack-reused 0
Receiving objects: 100% (75037/75037), 20.00 MiB | 27.09 MiB/s, done.
Resolving deltas: 100% (1/1), done.
Checking out files: 100% (75015/75015), done.


In [ ]:
%cd /content/bilingual_mnist

/content/bilingual_mnist


In [ ]:
IMAGE_PATH = '/content/bilingual_mnist/bilingual_data/'
LABELS = [0,1,2,3,4,5,6,7,8,9,10,100,1000,10000,100000000]
IMAGE_WIDTH = 28
IMAGE_HEIGHT = 28
IMAGE_CHANNELS = 1
RANDOM_STATE = 42
TEST_SIZE = 0.2
VAL_SIZE = 0.2
CONV_2D_DIM_1 = 16
CONV_2D_DIM_2 = 16
CONV_2D_DIM_3 = 32
CONV_2D_DIM_4 = 64
MAX_POOL_DIM = 2
KERNEL_SIZE = 3
BATCH_SIZE = 32
NO_EPOCHS_1 = 5
NO_EPOCHS_2 = 10
NO_EPOCHS_3 = 50
PATIENCE = 5
VERBOSE = 1


In [ ]:
os.listdir("/content/bilingual_mnist/bilingual_data/")

['100',
 '2',
 '9',
 '4',
 '8',
 '7',
 '.DS_Store',
 '1',
 '100000000',
 '10',
 '3',
 '0',
 '5',
 '6',
 '10000',
 '1000']

In [ ]:
X = []
Y = []
def assign_label(img,label):
    return label

def make_train_data(label,DIR):
    for img in os.listdir(DIR):
        label=assign_label(img,label)
        path = os.path.join(DIR,img)
        img = cv.imread(path,cv.IMREAD_COLOR)
        #img = cv.resize(img, (IMAGE_WIDTH,IMAGE_HEIGHT))
    
        X.append(np.array(img))
        Y.append(str(label))

for i in LABELS:
  make_train_data(str(i),IMAGE_PATH+str(i))

# USE A LABELENCODER IF YOU WANT HERE

holder = zip(X, Y)


In [ ]:
#split into training and test data; 80% and 20%
data_train, data_test, labels_train, labels_test = train_test_split(X, Y, test_size=TEST_SIZE, random_state=RANDOM_STATE)

In [ ]:
#split training into train and validation
data_train, data_val = train_test_split(data_train, test_size=VAL_SIZE, random_state=RANDOM_STATE)

In [ ]:
print("Train set rows: {}".format(len(data_train)))
print("Test  set rows: {}".format(len(data_test)))
print("Val   set rows: {}".format(len(data_val)))

In [ ]:
print("========================== Visualizing Data ==========================")
fig = plt.figure(figsize=(8, 8))
fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0.05, wspace=0.05)

# plot the digits: each image is 28x28 pixels
for i in range(64):
    ax = fig.add_subplot(8, 8, i + 1, xticks=[], yticks=[])
    ax.imshow(data_test[i], cmap=plt.cm.binary, interpolation='nearest')
    
    # label the image with the target value
    ax.text(0, 7, str(labels_test[i]))
plt.show()

In [ ]:
# Creating model
print("========================== Creating a model ==========================")
model = keras.models.Sequential([
                                 keras.layers.Conv2D(64,(3,3),activation='relu',input_shape=(28,28,1)),
                                 keras.layers.MaxPool2D(2,2),
                                 keras.layers.Flatten(),
                                 keras.layers.Dense(28,activation='relu'),
                                 keras.layers.Dense(10,activation='softmax')
])
print("")
print("============================ modelSummary ============================")
print(model.summary())

In [ ]:
# Model Compile
print("========================== Compiling  model ==========================")
model.compile(optimizer='adam',metrics=['acc'],loss='sparse_categorical_crossentropy')
print("")
print("========================== Model's Compiled ==========================")
# Accuracy Callback
class handwriting_acc_callback(keras.callbacks.Callback):
  def on_epoch_end(self,epoch,logs={}):
    if(logs.get('acc')>=0.9):
       print("\n Model has reached 90% accuracy! Congratulations !!!!!")
       self.model.stop_training = True

handwriting_acc_callback = handwriting_acc_callback()
print("")

In [ ]:
model.fit(data_train,labels_train,epochs=3,callbacks=[handwriting_acc_callback], validation_data=data_val
          )

In [ ]:
# Model Fitting
print("============================= Model- Fit =============================")
model.fit(data_train,labels_train,epochs=3,callbacks=[handwriting_acc_callback])
print("")
# Model Evaluation
print("========================== Model Evaluation ==========================")
model.evaluate(data_test,labels_test)
print("")
print("======================================================================")
print("======================================================================")
print("======= Congratulations model was succesful with >90% accuracy =======")
print("======================================================================")
print("======================================================================")
print("")
print("================= Now goahead and run the next cell! =================")